### Vector retiever

In [6]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
import os
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [8]:
from langchain_groq import ChatGroq

llm = ChatGroq(
           
            model_name="mistral-saba-24b",
            temperature=0.7,
            max_tokens=4096
        )
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x78e619b5bd10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x78e619b54e30>, model_name='mistral-saba-24b', model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=4096)

In [9]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    # Additional Documents
    Document(
        page_content="Rabbits are social animals that require mental stimulation and space to hop.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Turtles can live for decades and need a controlled aquatic environment.",
        metadata={"source": "reptile-pets-doc"},
    ),
    Document(
        page_content="Hamsters are nocturnal pets that enjoy burrowing and storing food.",
        metadata={"source": "mammal-pets-doc"},
    ),
]


In [10]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that require mental stimulation and space to hop.'),
 Document(metadata={'source': 'reptile-pets-doc'}, page_content='Turtles can live for decades and need a controlled aquatic environment.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Hamsters are nocturnal pets that enjoy burrowing and storing food.')]

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings
embeeds=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeeds


HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
from langchain_chroma import Chroma
vectorstore=Chroma.from_documents(documents,embedding=embeeds)
vectorstore

In [14]:
vectorstore.similarity_search("cat")
##async query
await vectorstore.asimilarity_search("cat")



[Document(id='883cc4ae-66f4-412c-82d3-53144357a1ec', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='6c467047-537a-417f-b087-392e35256196', metadata={'source': 'mammal-pets-doc'}, page_content='Hamsters are nocturnal pets that enjoy burrowing and storing food.'),
 Document(id='81acbce6-a6b6-4ce6-80bb-68df6420fc13', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that require mental stimulation and space to hop.'),
 Document(id='8a92f085-4fa2-483a-8c4e-f311c5eb4108', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [15]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='883cc4ae-66f4-412c-82d3-53144357a1ec', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351054430007935),
 (Document(id='6c467047-537a-417f-b087-392e35256196', metadata={'source': 'mammal-pets-doc'}, page_content='Hamsters are nocturnal pets that enjoy burrowing and storing food.'),
  1.4276583194732666),
 (Document(id='81acbce6-a6b6-4ce6-80bb-68df6420fc13', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that require mental stimulation and space to hop.'),
  1.526151180267334),
 (Document(id='8a92f085-4fa2-483a-8c4e-f311c5eb4108', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740900039672852)]

In [16]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='883cc4ae-66f4-412c-82d3-53144357a1ec', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='8a92f085-4fa2-483a-8c4e-f311c5eb4108', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [ ]:
##best method 
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)
retriever.batch(["cat","dog"])


[[Document(id='883cc4ae-66f4-412c-82d3-53144357a1ec', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='8a92f085-4fa2-483a-8c4e-f311c5eb4108', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

In [18]:
## RAG
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|llm

response=rag_chain.invoke("tell me about dogs")
print(response.content)


Dogs are great companions, known for their loyalty and friendliness.
